In [1]:
import pandas as pd 
import json 
import requests
import os 
from dotenv import load_dotenv
import sys 

def print_json(data:json):
    """json 형태의 데이터를 출력하는 함수입니다.

    Args:
        data (json): json 형태의 데이터
    """
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)

def find_target_directory(target_dir_name):
    current_path = os.getcwd()
    while True:
        # 상위 디렉토리에서 대상 디렉토리 이름을 찾음
        parent_path, current_dir = os.path.split(current_path)
        
        if current_dir == target_dir_name:
            return current_path
        
        if parent_path == current_path:  # 최상위 디렉토리에 도달하면 종료
            break
        
        # 상위 디렉토리로 이동
        current_path = parent_path
    
    return None  # 대상 디렉토리를 찾지 못한 경우


def get_project_root_path(proejct_directory_name: str='odysseyes'):
    return  find_target_directory(target_dir_name=proejct_directory_name)


project_root_path = get_project_root_path()
print(f"프로젝트 경로: {project_root_path}")
sys.path.append(os.path.join(project_root_path, 'recommend', 'func'))

from kakao_client import KakaoMobilityClient

프로젝트 경로: /Users/jaehwan/SSAFY/odysseyes


In [2]:
load_dotenv()
kakao_api_key = os.getenv('KAKAO_API_KEY')

In [3]:
kakao_mobility_client = KakaoMobilityClient(api_key=kakao_api_key)

In [4]:
start_poi = {'latitude': '36.29285019', 'longitude': '126.92453303', 'name': '무드빌리지'}
end_poi = {'latitude': '36.28912812', 'longitude': '126.90875677', 'name': '장원막국수'}

test_route = kakao_mobility_client.get_route_data(
    start_poi=start_poi,
    end_poi=end_poi
)

In [5]:
print_json(test_route)

{
    "trans_id": "01931b2fe00272589fc8c2dc9b157735",
    "routes": [
        {
            "result_code": 0,
            "result_msg": "\uae38\ucc3e\uae30 \uc131\uacf5",
            "summary": {
                "origin": {
                    "name": "",
                    "x": 126.92453039868052,
                    "y": 36.29284962721872
                },
                "destination": {
                    "name": "",
                    "x": 126.90875167542968,
                    "y": 36.28911978854367
                },
                "waypoints": [],
                "priority": "RECOMMEND",
                "bound": {
                    "min_x": 126.90807785571958,
                    "min_y": 36.2839496192783,
                    "max_x": 126.92578996661233,
                    "max_y": 36.292734778483485
                },
                "fare": {
                    "taxi": 6800,
                    "toll": 0
                },
                "distance": 2798,
         

In [9]:
import folium

# JSON 데이터 (예제 데이터)
route_data = test_route  # 위의 JSON 데이터 전체를 여기 넣어주세요

# 지도 설정 (출발지 기준으로 초기 설정)
start_lat = route_data['routes'][0]['summary']['origin']['y']
start_lon = route_data['routes'][0]['summary']['origin']['x']
m = folium.Map(location=[start_lat, start_lon], zoom_start=14)

# 출발지와 도착지 표시
end_lat = route_data['routes'][0]['summary']['destination']['y']
end_lon = route_data['routes'][0]['summary']['destination']['x']
folium.Marker([start_lat, start_lon], tooltip="출발지", icon=folium.Icon(color="green")).add_to(m)
folium.Marker([end_lat, end_lon], tooltip="도착지", icon=folium.Icon(color="red")).add_to(m)

# 경로 표시 (폴리라인)
polyline_points = list()
for section in route_data['routes'][0]['sections']:
    for road in section['roads']:
        vertexes = road['vertexes']
        # vertexes를 [위도, 경도] 리스트로 변환하여 PolyLine을 생성
        polyline_points.extend([(vertexes[i+1], vertexes[i]) for i in range(0, len(vertexes), 2)])
folium.PolyLine(locations=polyline_points, color="blue", weight=3).add_to(m)

print(polyline_points[0])
# 안내 지점 (guides) 추가
# for guide in route_data['routes'][0]['sections'][0]['guides']:
#     guide_lat = guide['y']
#     guide_lon = guide['x']
#     guide_text = guide['guidance']
#     folium.Marker([guide_lat, guide_lon], popup=guide_text, icon=folium.Icon(color="orange")).add_to(m)

# 결과 출력
m


(36.292722970622755, 126.92447647373666)
